In [1]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json

In [12]:
today = datetime.datetime.now()
yesterday = today - datetime.timedelta(days = 1)
yesterday

datetime.datetime(2026, 1, 9, 19, 25, 34, 923255)

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 

2026-01-10 19:20:03,707 INFO: Initializing external client
2026-01-10 19:20:03,708 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-10 19:20:05,413 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1296539


In [4]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="flight_xgboost_model",
    version=1,
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

2026-01-10 19:20:14,410 INFO: Initializing for batch retrieval of feature vectors


Downloading: 0.000%|          | 0/449985 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/121390 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/25038 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/17452 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/152799 elapsed<00:00 remaining<?

In [5]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score=[277.9605], booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'float',
                            'float', 'float', 'float', 'float'],
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

## How the pipeline should look like 

Idea is to get yesterdays wheater and calander data and predict the total number of flights that will occur today. 

In [16]:
# Retrieve the google trend values to predict tomorrows flights 
weather_cal_data_fg = fs.get_feature_group(
    name='stockholm_weather_calendar_features',
    version=1,
)

batch_data = weather_cal_data_fg.filter(weather_cal_data_fg.date == yesterday).read()
X = batch_data.drop(columns=['date'])
y_pred = retrieved_xgboost_model.predict(X)
X

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.85s) 


,day_of_week,is_weekend,week_of_year,month,is_holiday,tavg,prcp,snow,wspd


In [9]:
forecast_row = pd.DataFrame({
    "date":[tomorrow],
    "predicted_landings":[y_pred],
    "total_landings":[None],
})

forecast_row

,date,predicted_landings,total_landings
0,2026-01-11 19:20:03.040796,[],None


In [53]:
# Get or create feature group for storing the monitoring values 
monitor_fg = fs.get_or_create_feature_group(
    name='flight_prediction',
    description='Flight prediction monitoring',
    version=1,
    primary_key=['city']
    event_time= "date"
)

2026-01-09 15:24:51,506 ERROR: Parser Error: . Detail: Python exception: Traceback (most recent call last):
  File "/usr/src/app/src/server.py", line 142, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 166, in wrapper
    result = func(instance, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 196, in do_get
    return self._read_query(context, path, command)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 123, in wrapper
    return func(instance, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 131, in wrapper
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/src/app/src/server.py", line 227, in _read_query
    result_batches = self.hudi_query_engine.read_query(query_obj)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

FeatureStoreException: Could not read data using Hopsworks Query Service.

In [ ]:
monitor_fg.insert(forecast_row, wait=True)

In [48]:
# Retrieve feature group
flight_data_fg = fs.get_feature_group(
    name='flight_data_arlanda',
    version=1,
)

outcome_df = flight_data_fg.filter(flight_data_fg.date == today).read() # this give todays 

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.62s) 
